## 出力層の設計

* ニューラルネットワークは分類問題と回帰問題の両方に用いることができる
    * 分類問題
        * 分類など
        * 例
            * グラフ上にプロットしたデータをズバッと二つに線で分ける
            * 応用例としてはスパムメールフィルタ
                * スパムか、スパムでは無いか
        * 使う<font color="red">出力層の</font>活性化関数
            * <font color="red">ソフトマックス関数</font>
    * 回帰問題
        * 予想など
        * 例
            * データ群から線を求める
            * 株価の予想とか明日の天気の予想など
        * 使う出力層の活性化関数
            * 恒等関数

### 恒等関数とソフトマックス関数

* 恒等関数
    * 入力をそのまま出力する
    * 入ってきたものに対して何も手を加えずに出力する関数
* ソフトマックス関数
    * $ \displaystyle y_k=\frac{\exp(a_k)}{\sum_{i=1}^{n} \exp(a_i)} $
    * 出力層が全部で $n$ 個あるとする
    * $k$ 番目の出力は $y_k$
    * $a_k$ は入力信号
    * 分母は全ての入力信号の指数関数の和
        * 出力の各ニューロンが全ての入力信号の影響を受ける



In [13]:
import numpy  as np

a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a)
# print(exp_a)

sum_exp_a = np.sum(exp_a)
#print(sum_exp_a)

y = exp_a / sum_exp_a
#print(y)

### 関数として定義
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

print(softmax(a))

[ 0.01821127  0.24519181  0.73659691]


### ソフトマックス関数の実装上の注意

* オーバーフローに関する問題
    * オーバーフロー
        * 数には有効桁数が有り、とても大きな値は表現できない
            * 表現できる数値の範囲に限界がある
    * 指数関数の値が大きくなるとオーバーフローする
        * そのような大きな値同士で割り算を行うと数値が不安定な結果になってしまう
        
### ソフトマックス関数の実装改善

\begin{align*}
y_k &= \frac{\exp(a_k)}{\sum_{i=1}^{n} \exp(a_i)} \\
&= \frac{C \exp(a_k)}{C \sum_{i=1}^{n} \exp(a_i)} \\
&= \frac{\exp(a_k + \log C) }{\sum_{i=1}^{n} \exp(a_i + \log C)} \\
&= \frac{\exp(a_k + C') }{\sum_{i=1}^{n} \exp(a_i + C')}
\end{align*}

* $C$ という任意の定数を分母と分子にかけて指数関数の中に移動させて $C'$としている
    * <font color="red">分母と分子に同じ定数をかけているだけなので元の式は変更されていない</font>
    * $C'$ はどのような値を用いることもできるが、オーバーフロー対策としては入力信号の中で最大の値を用いる事が一般的

In [14]:
a = np.array([1010, 1000, 990])

### オーバーフローする
print(np.exp(a) / np.sum(np.exp(a)))

c = np.max(a)
print(a - c)

print(np.exp(a-c) / np.sum(np.exp(a-c)))

[ nan  nan  nan]
[  0 -10 -20]
[  9.99954600e-01   4.53978686e-05   2.06106005e-09]


/usr/local/pyenv/versions/3.6.1/envs/jupyter/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in exp
/usr/local/pyenv/versions/3.6.1/envs/jupyter/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide


In [15]:
a = np.array([20, 10, 5])
print(np.exp(a) / np.sum(np.exp(a)))

c = np.max(a)
print(a - c)

print(np.exp(a-c) / np.sum(np.exp(a-c)))

[  9.99954296e-01   4.53978548e-05   3.05888340e-07]
[  0 -10 -15]
[  9.99954296e-01   4.53978548e-05   3.05888340e-07]


### ソフトマックス関数の再実装

In [16]:
import numpy  as np

a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a)

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

y = softmax(a)
print(y)
print(np.sum(y))

[ 0.01821127  0.24519181  0.73659691]
1.0


### ソフトマックス関数の特徴

* 出力
    * 0〜1.0の間の実数になる
* 総和
    * 1になる
        * つまり、<font color="red">ソフトマックス関数の出力を「確率」として解釈できる</font>
        * y[0] = 1.8%, y[1] = 24.5%, y[2] = 73.7%,
            * y[2]がもっとも確率が高いため、答えは2番目のクラスという答え方ができる
            * それぞれの確率で各クラスになりえる、という確率的な答え方もできる
* ソフトマックス関数を適用しても各要素の大小関係は変わらない
    * 指数関数は単調増加関数であるため
    * ニューラルネットワークのクラス分類は一般的に<font color="red">出力の一番大きいニューロンに相当するクラスだけを認識結果とする</font>
        * ソフトマックス関数を適用しても出力の一番大きいニューロンの位置は変わらない
            * ニューラルネットワークによる分類では、出力層のソフトマックス関数は省略することができる
            * 指数関数の計算はそれなりにコンピュータ計算が必要になるため、出力層のソフトマックス関数は省略すのが一般的
    * 機械学習の「学習」と「推論」のフェーズにおいて、「推論」のフェーズではソフトマックス関数を省略するのが一般的


## 出力層のニューロンの数

* クラス分類を行う問題
    * 分類したいクラスの数に設定するのが一般的
    * 例
        * ある入力画像に対してそれが0〜９のどれかを予想する(10クラス分類問題)の場合、出力層のニューロンの数は10個に設定する

## 手書き数字認識

* 機械学習の問題を解く手順
    * 学習
        * 訓練データ(学習データ)を使って重みパラメータの学習を行う
    * 推論
        * 先に学習したパラメータを使って入力データの分類を行う

### MNISTデータセット

* MNISTデータセット
    * 手書き数字の画像セット
    * 機械学習の分野で最も有名なデータセットの一つ
    * 簡単な実験から論文として発表される研究まで様々な場所で利用される
        * 実際に論文でも実験用のデータとしてよく登場する
    * 訓練画像60000枚、テスト画像10000枚用意されている
        * 28x28のグレー画像(1チャンネル)で各ピクセルは0〜255までの値を取る
        * それぞれの画像データに対しては数値に対応するラベルが与えられている

### PythonスクリプトによるMNISTデータセットの確認

* mnist.py
    * やること
        * データセットのダウンロード
        * 画像データのNumPy配列変換
    * load_mnist関数
        * 読み込み
            * 1回目 : ネット上からMNISTデータをダウンロード
            * 2回目以降 : ローカルに保存したpickleファイルを読み込む
        * 返す値
            * x_train : 訓練画像
            * t_train : 訓練ラベル
            * x_test : テスト画像
            * t_test : テストラベル
        * 引数
            * normalize
                * 入力画像を0.0〜1.0に正規化するかを選択
                    * Falseだと入力画像のピクセルは0〜255のまま
            * flatten
                * 入力画像を平らにする(1次元配列にする)かどうかを設定
                    * True : 784個の要素からなる1次元配列として格納される
                    * False : 1 x 28 x 28の3次元配列として格納される
            * one_hot_lavel
                * ラベルをone_hotとして格納するかどうかを設定する
                    * one_hotラベル
                        * 正解となるラベルが1でそれ以外は0の配列
                        * 例) [0,0,1,0,0,0,1,1,0] 
                * True : one_hot1ラベル
                * False : 7 ,2　のように単純に正解のラベルが格納される

In [17]:
import sys, os

#sys_path = '/Users/ftakao2007/jupyter/deep-learning-from-scratch'
sys_path = '/root/jupyter/deep-learning-from-scratch'
if not sys_path in sys.path:
    sys.path.append(append_sys_path)

from dataset.mnist import load_mnist
print(sys.path)

['', '/usr/local/pyenv/versions/3.6.1/lib/python36.zip', '/usr/local/pyenv/versions/3.6.1/lib/python3.6', '/usr/local/pyenv/versions/3.6.1/lib/python3.6/lib-dynload', '/usr/local/pyenv/versions/3.6.1/envs/jupyter/lib/python3.6/site-packages', '/usr/local/pyenv/versions/3.6.1/envs/jupyter/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython', '/root/jupyter/deep-learning-from-scratch']


In [18]:
(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False)
    
#print(x_train)
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [19]:
import sys, os
#sys_path = '/Users/ftakao2007/jupyter/deep-learning-from-scratch'
sys_path = '/root/jupyter/deep-learning-from-scratch'
if not sys_path in sys.path:
    sys.path.append(sys_path)

import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    # PIL用オブジェクトに変換
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)  # 5

print(img.shape)  # (784,)  入力画像を784個の要素からなる1次元配列に格納
img = img.reshape(28, 28)  # 形状を元の画像サイズに変形
print(img.shape)  # (28, 28)

### 画像の表示(ローカル実行のみ)
img_show(img)

5
(784,)
(28, 28)


In [20]:
img = x_train[1]
label = t_train[1]
print(label)  # 0

print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 形状を元の画像サイズに変形
print(img.shape)  # (28, 28)

### 画像の表示(ローカル実行のみ)
img_show(img)

0
(784,)
(28, 28)


In [21]:
img = x_train[2]
label = t_train[2]
print(label)  # 4

print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 形状を元の画像サイズに変形
print(img.shape)  # (28, 28)

### 画像の保存
Image.fromarray(np.uint8(img)).save('img/text_img_numpy.jpg', 'JPEG', quality=100, optimize=True)

4
(784,)
(28, 28)


画像の表示

<img src="img/text_img_numpy.jpg" alt="numpy画像の保存" title="numpy画像の保存" align="left" />  

<br clear="left">

### ニューラルネットワークの推論処理

* 推論処理を行うニューラルネットワークを実装
    * 入力層
        * 784個のニューロン
            * 画像サイズ 28 x 28 = 784から
    * 出力層
        * 10個のニューロン
            * 10クラス分類(0〜9の数字)
    * 隠れ層1
        * 50個のニューロン
        * 任意の値
    * 隠れ層2
        * 100個のニューロン
        * 任意の値


In [25]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

# 学習済みの重みパラメータ(重みとバイアスのパラメータがディクショナリ型の変数として保存されている)
pkl = "/root/jupyter/deep-learning-from-scratch/ch03/sample_weight.pkl"

def init_network():
    with open(pkl, "rb") as f:
        network = pickle.load(f)
        
    return network

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [27]:
import pickle

x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1
        
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
